# 第6回演習課題

In [1]:
import numpy as np
import theano.tensor as T
import theano

## 課題1．convとdownsampleの使い方

### Convolution

- 畳み込みのフィルタ（重み）$W_{i,j}^{k,l}$
    - 次元数4$(k,l,i,j)$
        - $l$：入力のチャネル数
        - $k$：フィルタ数（出力のチャネル数)
        - $i$：フィルタの行数
        - $j$：フィルタの列数
    - ストライド：フィルタを適用する位置の間隔（theanoのsubsampleオプション）
    - ゼロパディング：入力の周りに値0の縁を加える（theanoのborder_mode="full"オプション）
        - 入力のサイズを保つ為，フィルタの縦or横の次元が$F$のときパディング数を$(F-1)/2$とする．
        - ただしborder_mode="full"だと，$F-1$となることに注意
- 入力または隠れ層$X_{i,j}^{k}$
    - 次元数4$(n,k,i,j)$
        - $n$：バッチサイズ
        - $k$：チャネル数
        - $i$：入力の行数
        - $j$：入力の列数
- フィルタ後のサイズは，入力の縦or横の次元数$N$，フィルタの縦or横の次元数$F$，ストライドの縦or横の量$S$で決まる．
    - $(N-F)/S+1$
    - border_mode="full"の場合，S=1のとき$(N-F+2(F-1))+1=N+F-1$

In [2]:
from theano.tensor.nnet import conv

#入力 (バッチサイズとチャネル数は1）
x = T.fmatrix('x')
x_4d = x[np.newaxis,np.newaxis,:,:]
#x_4d = x.dimshuffle('x','x',0,1)でも可

#フィルタ(1,1,3,1)
W = np.array([[1,0,1],[0,1,0],[1,0,1]]).astype("float32").reshape(1, 1, 3, 3)

#畳み込み((5-3)+1=3)
convoluted_image = conv.conv2d(x_4d, W, border_mode="valid")

#ストライド(2×2)((5-3)/2+1=2)
stride_convoluted_image = conv.conv2d(x_4d, W, border_mode="valid",subsample=(2, 2))

#パディング(full)((5+3)-1=7)
fullpadding_convoluted_image = conv.conv2d(x_4d, W, border_mode="full")

#パディング(same size)
pd_h = W.shape[2]-1
pd_w = W.shape[3]-1
x_h = x_4d.shape[2]
x_w = x_4d.shape[3]
samepadding_convoluted_image = conv.conv2d(x_4d, W, border_mode="full")[:,:,pd_h:x_h+pd_h,pd_w:x_w+pd_w]

#Convolution Function
convolution = theano.function([x], convoluted_image)
stride_convolution = theano.function([x], stride_convoluted_image)
fullpadding_convolution = theano.function([x], fullpadding_convoluted_image)
samepadding_convolution = theano.function([x], samepadding_convoluted_image)

#Sample Image (5×5)
sample_image = np.array([[1., 1., 1., 0., 0.], 
                         [0., 1., 1., 1., 0.], 
                         [0., 0., 1., 1., 1.], 
                         [0., 0., 1., 1., 0.], 
                         [0., 1., 1., 0., 0.]]).astype("float32")

#Original Image
print sample_image

#Convolved Image
print convolution(sample_image).reshape(3, 3)
print stride_convolution(sample_image).reshape(2, 2)
print fullpadding_convolution(sample_image).reshape(7, 7)
print samepadding_convolution(sample_image).reshape(5, 5)

[[ 1.  1.  1.  0.  0.]
 [ 0.  1.  1.  1.  0.]
 [ 0.  0.  1.  1.  1.]
 [ 0.  0.  1.  1.  0.]
 [ 0.  1.  1.  0.  0.]]
[[ 4.  3.  4.]
 [ 2.  4.  3.]
 [ 2.  3.  4.]]
[[ 4.  4.]
 [ 2.  4.]]
[[ 1.  1.  2.  1.  1.  0.  0.]
 [ 0.  2.  2.  3.  1.  1.  0.]
 [ 1.  1.  4.  3.  4.  1.  1.]
 [ 0.  1.  2.  4.  3.  3.  0.]
 [ 0.  1.  2.  3.  4.  1.  1.]
 [ 0.  0.  2.  2.  1.  1.  0.]
 [ 0.  1.  1.  1.  1.  0.  0.]]
[[ 4.  3.  4.  1.  1.]
 [ 2.  4.  3.  3.  0.]
 [ 2.  3.  4.  1.  1.]
 [ 2.  2.  1.  1.  0.]
 [ 1.  1.  1.  0.  0.]]


### Pooling

- プーリングには次の種類がある
    - Max pooling (theanoでは'max')
    - Sum pooling (theanoでは'sum')
    - Mean pooling (theanoでは'average_exc_pad')
    - その他Lpプーリングなど(theano未実装)
- Convと同様，ストライドやパディングも考えることもある．
    - ストライドはデフォルトではdsと同じ
- ignore_border=Falseにすると，画像領域を超える

In [3]:
from theano.tensor.signal import downsample

#入力 (バッチサイズとチャネル数は1）
x = T.fmatrix('x')
x_4d = x[np.newaxis,np.newaxis,:,:]
#x_4d = x.dimshuffle('x','x',0,1)でも可

#pooling
pooled = downsample.max_pool_2d(input=x_4d, ds=(2,2), ignore_border=True)

#ストライド(1×1，デフォルトではdsと同じ）
stride_pooled = downsample.max_pool_2d(input=x_4d, ds=(2,2), st=(1,1), ignore_border=True)

#パディング
padding_pooled = downsample.max_pool_2d(input=x_4d, ds=(2,2), ignore_border=True, padding=(1,1))

#mean pooling
#mean_pooled = downsample.max_pool_2d(input=x_4d, ds=(2,2), mode='average_exc_pad', ignore_border=True)

#Pooling Function
pooling = theano.function([x], pooled)
stride_pooling = theano.function([x], stride_pooled)
padding_pooling = theano.function([x], padding_pooled)
#mean_pooling = theano.function([x], mean_pooled)

#Sample Image (5×5)
sample_image = np.array([[77, 80, 82, 78, 70], 
                         [83, 78, 80, 83, 82], 
                         [87, 82, 81, 80, 74], 
                         [87, 87, 85, 77, 66], 
                         [84, 79, 77, 78, 76]]).astype("float32")

print pooling(sample_image).reshape(2, 2)
print stride_pooling(sample_image).reshape(4, 4)
print padding_pooling(sample_image).reshape(3, 3)
#print mean_pooling(sample_image).reshape(2, 2)

[[ 83.  83.]
 [ 87.  85.]]
[[ 83.  82.  83.  83.]
 [ 87.  82.  83.  83.]
 [ 87.  87.  85.  80.]
 [ 87.  87.  85.  78.]]
[[ 77.  82.  78.]
 [ 87.  82.  83.]
 [ 87.  87.  78.]]


## 課題2．Conv layerとPooling layerの実装

Conv layer

In [4]:
from theano.tensor.nnet import conv
class Conv:
    def __init__(self,filter_shape,function,border_mode="valid",subsample=(1, 1)):
        
        self.function = function
        self.border_mode = border_mode
        self.subsample = subsample
        
        fan_in = np.prod(filter_shape[1:])
        fan_out = (filter_shape[0] * np.prod(filter_shape[2:]))
        
        self.W = theano.shared(rng.uniform(
                    low=-4*np.sqrt(6. / (fan_in + fan_out)),
                    high=4*np.sqrt(6. / (fan_in + fan_out)),
                    size=filter_shape
                ).astype("float32"),name="W")
        #バイアスはフィルタごと
        self.b = theano.shared(np.zeros((filter_shape[0],), dtype="float32"),name="b")
        self.params = [self.W,self.b]
        
    def fprop(self,x):
        #畳込み処理
        conv_out = conv.conv2d(x,self.W,
                               border_mode=self.border_mode,
                               subsample=self.subsample)
        #バイアスを加えて（第1要素）活性化関数をかける
        y = self.function(conv_out + self.b[np.newaxis,:,np.newaxis,np.newaxis])
        return y

Pooling layer

In [5]:
from theano.tensor.signal import downsample
class Pooling:
    def __init__(self,pool_size=(2,2)):
        self.pool_size=pool_size
        self.params = []
    def fprop(self,x):
        #プーリングした値を返す
        return downsample.max_pool_2d(x,self.pool_size,ignore_border=True)

Flatten layer

In [6]:
class Flatten:
    def __init__(self,outdim=2):
        self.outdim = outdim
        self.params = []
    def fprop(self,x):
        #flattenはoutdim次元にする関数
        return T.flatten(x,self.outdim)

## 宿題．畳み込みニューラルネットワークの実装，MNISTでの実験．

- データはmnist_x,mnist_yで与えられます
    - mnsit_xとmnist_yをtrain_X,train_yとvalid_X,valid_yに分けるなどしてモデルを学習してください
- test関数を定義してください
    - 採点システム側で用意したtest_Xを与えたときの出力の精度(F値)で評価します

In [13]:
import numpy as np
import theano
import theano.tensor as T
from theano.tensor.shared_randomstreams import RandomStreams
from collections import OrderedDict
rng = np.random.RandomState(1234)

%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split

from sklearn.metrics import f1_score
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')
mnist_x, mnist_y = mnist.data.astype("float32")/255.0, mnist.target.astype("int32")

次のセルを完成させて提出してください
- レイヤークラスなど，必要なものは全て書いてください

In [14]:
#SGD
def sgd(params,gparams,lr=0.1):
    updates = OrderedDict()
    for param, gparam in zip(params, gparams):
        updates[param] = (param - lr * gparam).astype("float32")
    return updates

#Layer 
class Layer:
    def __init__(self, in_dim, out_dim, function):
        ## WRITE ME Done
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.function = function
        self.W = theano.shared(
                                rng.uniform(
                                            low=-0.08, 
                                            high=0.08, 
                                            size=(in_dim, out_dim)
                                ).astype("float32"), name="W")
        self.b =  theano.shared(np.zeros(out_dim).astype("float32"), name="bias")
        
        self.params = [ self.W, self.b ]

    def fprop(self, x):
        ## WRITE ME Done
        h = self.function(T.dot(x, self.W)+self.b)
        self.h = h
        return self.h

    
#Conv layer
from theano.tensor.nnet import conv
class Conv:
    def __init__(self,filter_shape,function,border_mode="valid",subsample=(1, 1)):
        
        self.function = function
        self.border_mode = border_mode
        self.subsample = subsample
        
        fan_in = np.prod(filter_shape[1:])
        fan_out = (filter_shape[0] * np.prod(filter_shape[2:]))
        
        self.W = theano.shared(rng.uniform(
                    low=-4*np.sqrt(6. / (fan_in + fan_out)),
                    high=4*np.sqrt(6. / (fan_in + fan_out)),
                    size=filter_shape
                ).astype("float32"),name="W")
        #バイアスはフィルタごと
        self.b = theano.shared(np.zeros((filter_shape[0],), dtype="float32"),name="b")
        self.params = [self.W,self.b]
        
    def fprop(self,x):
        #畳込み処理
        conv_out = conv.conv2d(x,self.W,
                               border_mode=self.border_mode,
                               subsample=self.subsample)
        #バイアスを加えて（第1要素）活性化関数をかける
        y = self.function(conv_out + self.b[np.newaxis,:,np.newaxis,np.newaxis])
        return y

#Pooling layer
from theano.tensor.signal import downsample
class Pooling:
    def __init__(self,pool_size=(2,2)):
        self.pool_size=pool_size
        self.params = []
    def fprop(self,x):
        #プーリングした値を返す
        return downsample.max_pool_2d(x,self.pool_size,ignore_border=True)
#Flatten layer
class Flatten:
    def __init__(self,outdim=2):
        self.outdim = outdim
        self.params = []
    def fprop(self,x):
        #flattenはoutdim次元にする関数
        return T.flatten(x,self.outdim)

print "Setting up data...",
train_X, valid_X, train_y, valid_y = train_test_split(mnist_x, mnist_y, test_size=0.2, random_state=42)
print "Done"

print "Setting up layers...",
activation = T.nnet.sigmoid
layers = [
    Conv((20,1,5,5),activation),
    Pooling((2,2)),
    Conv((50,20,5,5),activation),
    Pooling((2,2)),
    Flatten(2),
    Layer(800,500, activation),#800=((((28-5+1)/2)-5+1)/2)**2*50
    Layer(500,10, T.nnet.softmax)
]
print "Done"

x, t = T.fmatrix("x"), T.ivector("t")
x_4d = x.reshape((x.shape[0],1,28,28)) #画像を4次元にする

params = []
layer_out = x_4d
for i, layer in enumerate(layers):
    params += layer.params
    layer_out = layer.fprop(layer_out)

y = layers[-1].h
cost = - T.mean((T.log(y))[T.arange(x.shape[0]), t])

gparams = T.grad(cost, params)
updates = sgd(params,gparams)

train = theano.function([x,t], cost, updates=updates)
valid  = theano.function([x,t],[cost, T.argmax(y, axis=1)])
test  = theano.function([x],T.argmax(y, axis=1))

##以下は通常のMLPと同じ実装で訓練
print "Start Training"
batch_size = 100
nbatches = train_X.shape[0]//batch_size
for epoch in range(50):
    ## WRITE ME
    train_X, train_y = shuffle(train_X, train_y)  # Shuffle Samples !!
    for i in range(nbatches):
            start = i * batch_size
            end = start + batch_size
            train(train_X[start:end], train_y[start:end])
    valid_cost, pred = valid(valid_X, valid_y)

    print "EPOCH:: %i, Validation cost: %.3f, Validation F1: %.3f"%(epoch+1, valid_cost, f1_score(valid_y, pred, average="macro"))

Setting up data... Done
Setting up layers...
Done


TypeError: ('An update must have the same type as the original shared variable (shared_var=W, shared_var.type=TensorType(float32, 4D), update_val=Elemwise{sub,no_inplace}.0, update_val.type=TensorType(float64, 4D)).', 'If the difference is related to the broadcast pattern, you can call the tensor.unbroadcast(var, axis_to_unbroadcast[, ...]) function to remove broadcastable dimensions.')

以下の処理は，システム側で行います

In [ ]:
pred_y = test(test_X)